# chat UI in fastHTML

> chat UI implemented in fastHTML

In [ ]:
#| default_exp chat_ui

In [ ]:
#| hide
from nbdev.showdoc import *

## Chat App initialization

Start by creating the chat application with `FastHTML`.

In [ ]:
#| export
import uvicorn
import importlib.util
from fasthtml.common import *
from llmcam.fn_to_fc import capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file
from llmcam.fn_to_fc import tool_schema, complete, form_msg
from llmcam.store import add_api_tools, add_function_tools, remove_tools
from llmcam.store import execute_handler_core, initialize_handlers

In [ ]:
#| export
# Set up base youtube live tools and session tools as simplified database
YTLiveTools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file)]
session_tools = {}
def execute_handler(function_name, **kwargs):
    return execute_handler_core(session_tools, function_name, **kwargs)

In [ ]:
#| export
# Set up the app, including daisyui and tailwind for the chat component
hdrs = (picolink, Script(src="https://cdn.tailwindcss.com"),
        Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@4.11.1/dist/full.min.css"),
        MarkdownJS(), HighlightJS(langs=['python', 'javascript', 'html', 'css']))
app = FastHTML(hdrs=hdrs)

## Chat components

Basic chat UI components can include Chat Message and a Chat Input. For a Chat Message, the important attributes are the actual message (str) and the role of the message owner (user - boolean value whether the owner is the user, not the AI assistant).

In [ ]:
#| export
# Chat message component (renders a chat bubble)
def ChatMessage(
        msg: str,  # Message to display
        user: bool  # Whether the message is from the user or assistant
    ):  # Returns a Div containing the chat bubble
    # Set class to change displayed style of bubble
    bubble_class = "chat-bubble-primary" if user else 'chat-bubble-secondary'
    chat_class = "chat-end" if user else 'chat-start'
    return  Div(cls=f"chat {chat_class}")(
                Div('User' if user else 'Assistant', cls="chat-header"),
                Div(
                    msg,
                    cls=f"chat-bubble {bubble_class} marked px-6 py-4", 
                    style=f"background-color: {'#038a5e' if user else '#025238'}; color: {'black' if user else 'white'};"),
                Hidden(msg, name="contents"),  # Hidden field for submitting past contents to form
                Hidden("user" if user else "assistant", name="roles")  # Hidden field for submitting corresponding owners
            )

In [ ]:
show_doc(ChatMessage)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L27){target="_blank" style="float:right; font-size:smaller"}

### ChatMessage

>      ChatMessage (msg:str, user:bool)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| msg | str | Message to display |
| user | bool | Whether the message is from the user or assistant |

For the chat input, set the name for submitting a new message via form.

In [ ]:
#| export
# The input field for the user message. Also used to clear the
# input field after sending a message via an OOB swap
def ChatInput():  # Returns an input field for the user message
    return Input(name='msg', id='msg-input', placeholder="Type a message",
                 cls="input input-bordered w-full rounded-l-2xl bg-stone-800", 
                 hx_swap_oob='true'  # Re-render the element to remove submitted message
                )

In [ ]:
show_doc(ChatInput)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L47){target="_blank" style="float:right; font-size:smaller"}

### ChatInput

>      ChatInput ()

### Action Buttons

Simple actions for creating a new message from the user side.

In [ ]:
#| export
def ActionButton(
        content: str  # Text to display on the button
    ):  # Returns a button with the given content

    return Form(
        hx_post="/",
        hx_target="#chatlist",
        hx_swap="beforeend",  # Location: just before the end of element
    )(
        Hidden(content, name="msg"),
        Button(content, cls="btn btn-secondary")
    )

def ActionPanel():  # Returns a panel of action buttons
    return Div(
        ActionButton("Introduce your model GPT-4o"),
        ActionButton("Extract information from a YouTube Live"),
        cls="flex flex-row h-fit px-24 gap-4 pt-4"
    )

In [ ]:
show_doc(ActionButton)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L54){target="_blank" style="float:right; font-size:smaller"}

### ActionButton

>      ActionButton (content:str)

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| content | str | Text to display on the button |

In [ ]:
show_doc(ActionPanel)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L67){target="_blank" style="float:right; font-size:smaller"}

### ActionPanel

>      ActionPanel ()

## Router

### Home page
The home page should contain our message list and the Chat Input. The main page can be extracted by accessing the index (root) endpoint.

In [ ]:
#| export
scroll_script = Script("""
  // Function to scroll to the bottom of an element
  function scrollToBottom(element) {
    element.scrollTop = element.scrollHeight;
  }

  // Reference the expanding element
  const expandingElement = document.getElementById('chatlist');

  // Observe changes to the element's content and scroll down automatically
  const observer = new MutationObserver(() => {
    scrollToBottom(expandingElement);
  });

  // Start observing the expanding element for changes
  observer.observe(expandingElement, { childList: true, subtree: true });
""")

In [ ]:
#| export
@app.get('/')
def index():
    sidebar = Div(
        H1("Conversations"),
        cls="w-[30vw] bg-stone-800"
    )
    page =  Div(cls="w-full flex flex-col p-0")(
        ActionPanel(),
        Form(
            hx_post="/",  # Operation: some POST endpoint with function `send` 
            hx_target="#chatlist",  # Target: element with ID 'chatlist'
            hx_swap="beforeend",  # Location: just before the end of element
            cls="w-full flex flex-col px-24 h-[90vh]"
        )(
            # The chat list
            Div(id="chatlist", cls="chat-box overflow-y-auto flex-1 w-full mt-10")(
                # One initial message from AI assistant
                ChatMessage("Hello! I'm a chatbot. How can I help you today?", False),
            ),
            # Input form
            Div(cls="h-fit mb-5 mt-5 flex space-x-2 mt-2")(
                Group(
                    ChatInput(), 
                    Button("Send", cls="btn btn-primary rounded-r-2xl", style="background-color: #03fcad;"))
            ),
            scroll_script
        )   
    )
    return Main(
        sidebar,
        page, 
        data_theme="forest", 
        cls="h-[100vh] w-full relative flex flex-row items-stretch overflow-hidden transition-colors z-0 p-0",)

### Form submission

At submission, this function should:

- Extract the new and all previous chat history  
- Prompt & get answers from ChatGPT from all these messages  
- Return a new ChatMessage

In [ ]:
#| export

# Handle the form submission
@app.post('/')
def send(session, msg: str, contents: list[str] = None, roles: list[str] = None):
    if "session_id" not in session:
        # Initialize tools in session tools and create a session ID
        session_id, tools = initialize_handlers(
            session_tools=session_tools,
            functions=[add_api_tools, add_function_tools, remove_tools], 
            service_name="toolbox_handler",
            fixup=execute_handler
        )
        session["session_id"] = session_id
        tools.extend(YTLiveTools)
    else:
        # Retrieve toolbox via session ID
        session_id = session["session_id"]
        tools = session_tools[session_id]

    # If no contents or roles are provided, set them to empty lists
    if not contents: contents = []
    if not roles: roles = []

    # Create chat messages from the provided contents and roles
    messages = [ form_msg(role, content) for role, content in zip(roles, contents) ]
    nof_old_msgs = len(messages) # Number of old messages
    messages.append(form_msg("user", msg))
    
    # Add the user's message to the chat history
    complete(messages, tools)
    responses = messages[nof_old_msgs:]  # Get only the new messages
    
    # Create chat messages from the responses
    chat_messages = [
        ChatMessage(res['content'], res['role'] == 'user') for res in responses if 'content' in res \
            if res['role'] in ['user', 'assistant'] and res['content'] is not None
    ]
    
    return (*chat_messages,
            ChatInput()) # And clear the input field via an OOB swap

## Runner

In addition to the main app, an utility function is implemented to run the app just by importing and executing this function to a Python file.

In [ ]:
#| export
def llmcam_chatbot(
        package_name="ninjalabo.llmcam",  # The installed package name
        module_name="chat_ui",  # The module containing the FastAPI app
        app_variable="app",  # The FastAPI app variable name
        host="0.0.0.0",  # The host to listen on
        port=5001,  # The port to listen on
        **uvicorn_kwargs  # Additional keyword arguments for uvicorn
    ):
    "Find and run the FastAPI app in the specified module within the given package."
    # Construct the full module path (e.g., 'llmcam.chat_ui')
    full_module_path = f"{package_name.split('.')[-1]}.{module_name}"

    # Check if the module exists in the installed package
    try:
        spec = importlib.util.find_spec(full_module_path)
        if spec is None:
            print(f"Module '{full_module_path}' not found in package '{package_name}'.")
            return
        # Dynamically run the Uvicorn server
        uvicorn.run(f"{full_module_path}:{app_variable}", host=host, port=port, **uvicorn_kwargs)
    except Exception as e:
        print(f"Error running the app: {e}")

In [ ]:
show_doc(llmcam_chatbot)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/chat_ui.py#L155){target="_blank" style="float:right; font-size:smaller"}

### llmcam_chatbot

>      llmcam_chatbot (package_name='ninjalabo.llmcam', module_name='chat_ui',
>                      app_variable='app', host='0.0.0.0', port=5001,
>                      **uvicorn_kwargs)

*Find and run the FastAPI app in the specified module within the given package.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| package_name | str | ninjalabo.llmcam | The installed package name |
| module_name | str | chat_ui | The module containing the FastAPI app |
| app_variable | str | app | The FastAPI app variable name |
| host | str | 0.0.0.0 | The host to listen on |
| port | int | 5001 | The port to listen on |
| uvicorn_kwargs |  |  |  |

For running while testing with Jupyter notebook, use the `JupyUvi` in `fasthtml` to run in separate thread.

In [ ]:
#| eval: false
from fasthtml.jupyter import *

server = JupyUvi(app=app)

In [ ]:
#| eval: false
server.stop()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()